In [1]:
# pip install pdfminer.six
# pip install PyMuPDF
# pip install pandas
# pip install pillow
# pip install matplotlib
# pip install transformers
# pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html
# pip install accelerate
# pip install https://github.com/jllllll/bitsandbytes-windows-webui/releases/download/wheels/bitsandbytes-0.41.0-py3-none-win_amd64.whl
# pip install langchain

In [2]:
from PDFextractor import PDFextractor
pdfextractor = PDFextractor()
text_results, image_results = pdfextractor.get_text_queue(r"C:\Users\Sagar\Python files_Jupiter\Git Repo Local\PDFChat\7_Phishing-Alarm_Robust_and_Efficient_Phishing_Detection_via_Page_Component_Similarity.pdf")

In [3]:
from VectorStore import VectorStore
vectorstore = VectorStore()

c:\Users\Sagar\Python files_Jupiter\Git Repo Local\PDFChat\PDFchat\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
vectorstore.add_vectors(text_results)

In [5]:
# model name
name = "MBZUAI/LaMini-GPT-1.5B"

In [6]:
auth_token = ""

In [7]:
# Import tranformer classes
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from transformers import BitsAndBytesConfig
import torch

In [8]:
# Create quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
)

# Create tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    name,
    cache_dir="./model/",
    token=auth_token
)

# Create model
model = AutoModelForCausalLM.from_pretrained(
    name,
    cache_dir="./model/",
    token=auth_token,
    device_map="auto",
    quantization_config=quantization_config
)

# Setup text streamer
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_token=True)

bin c:\Users\Sagar\Python files_Jupiter\Git Repo Local\PDFChat\PDFchat\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [9]:
prompt = "Summarize this article"

In [10]:
query_pdf = vectorstore.find_similar_vectors(prompt, 2)

In [11]:
query_information = ""
for info in query_pdf:
    query_information = query_information + info

In [12]:
extracted_prompt = query_information + prompt

In [13]:
inputs = tokenizer(extracted_prompt, return_tensors="pt").to(model.device)

In [14]:
# Actually run the thing
answer = model.generate(**inputs, streamer=streamer, use_cache=True, max_new_tokens=float("inf"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


:
The article presents two algorithms for evaluating the visual similarity of web pages. The first algorithm, called Match Score, measures how much visual appearance two pages have in common. The second algorithm, called Similarity Score, measures how similar two pages are to each other. Both algorithms are based on visual characteristics extracted from web pages.<|endoftext|>
